In [84]:
#Import dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import requests
from bs4 import BeautifulSoup
import re
import config
import json
import os
from urllib.parse import urlparse
import csv

In [85]:
#specify the path to chromedriver.exe (download and save on your computer)
driver = webdriver.Chrome()

#open the webpage
driver.get("https://www.instagram.com/")

In [86]:
#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#enter username and password
username.clear()
# username.send_keys(config.username)
# Add your fake username of Insta
username.send_keys("athena_ladakhi")
password.clear()
# password.send_keys(config.password)
password.send_keys("eco@1234")

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [87]:
time.sleep(5)
not_now_save = driver.find_element(By.XPATH, "/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div/div/div/div")
not_now_save.click()

time.sleep(3)

not_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//button[contains(text(), "Not Now")]')))

not_button.click()

In [88]:
# Wait up to 10 seconds for the search button to be clickable on the web page
search_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[aria-label="Search"]')))

# Click the search button once it becomes clickable
search_button.click()

In [89]:
#target the search input field
searchbox = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, "//input[@placeholder='Search']")))
searchbox.clear()

#search for the @handle or keyword
keyword = "@jlo"
searchbox.send_keys(keyword)

In [90]:
# Check if the keyword starts with "@"
time.sleep(2)
if keyword.startswith("@"):
    # Remove the "@" symbol
    keyword = keyword[1:]
    
# Find the first element with the specified XPath that matches the keyword    
first_result = driver.find_element(By.XPATH, f'//span[text()="{keyword}"]')

# Click on the found element (assuming it represents the desired search result)
first_result.click()

In [91]:
# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")

# Create a list to store htmls
soups = []

while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for a moment to allow new content to load (adjust as needed)
    time.sleep(5)
    
    # Parse the HTML
    html = driver.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(BeautifulSoup(html, 'html.parser'))

    # Get the current page height
    current_height = driver.execute_script("return document.body.scrollHeight")

    if current_height == initial_height:
        break  # Exit the loop when you can't scroll further

    initial_height = current_height  # Update the initial height for the next iteration

KeyboardInterrupt: 

In [ ]:
# List to store the post image URLs
post_urls = []

for soup in soups:
    # Find all anchor elements with href attributes
    anchors = soup.find_all('a', href=True)
    
    # Filter URLs that start with "/p/" or "/reel/"
    post_urls.extend([anchor['href'] for anchor in anchors if anchor['href'].startswith(("/p/", "/reel/"))])

# Convert the list to a set to remove duplicates
unique_post_urls = list(set(post_urls))

print(f"before: {len(post_urls)}, after: {len(unique_post_urls)}")

before: 170, after: 58


In [ ]:
json_list = []

# Define the query parameters to add
# query_parameters = "__a=1&__d=dis"
query_parameters = "__a=1&__d=dis"

# go through all urls
for url in unique_post_urls:
    try:
        # Get the current URL of the page
        current_url = driver.current_url

        # Append the query parameters to the current URL
        modified_url = "https://www.instagram.com/" + url + "?" + query_parameters

        # Get URL
        driver.get(modified_url)

        # Wait for a moment to allow new content to load (adjust as needed)
        time.sleep(1)

        # Find the <pre> tag containing the JSON data
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//pre'))
        )
        pre_tag = driver.find_element_by_xpath('//pre')

        # Extract the JSON data from the <pre> tag
        json_script = pre_tag.text

        # Parse the JSON data
        json_parsed = json.loads(json_script)

        # Add json to the list
        json_list.append(json_parsed)
    except (NoSuchElementException, TimeoutException, json.JSONDecodeError) as e:
        print("Error Parsing")
        # print(f"Error processing URL {url}: {e}")


AttributeError: 'WebDriver' object has no attribute 'find_element_by_xpath'

In [ ]:
# Lists to store URLs and corresponding dates
all_urls = []
all_dates = []

# Iterate through each JSON data in the list
for json_data in json_list:
    
    # Extract the list from the 'items' key
    item_list = json_data.get('items', [])
    
    # Iterate through each item in the 'items' list
    for item in item_list:
        
        # Extract the date the item was taken
        date_taken = item.get('taken_at')  # Move this line inside the loop

        # Check if 'carousel_media' is present
        carousel_media = item.get('carousel_media', [])
        
        # Iterate through each media in the 'carousel_media' list
        for media in carousel_media:
            
            # Extract the image URL from the media
            image_url = media.get('image_versions2', {}).get('candidates', [{}])[0].get('url')
            
            if image_url:
                # Add the image URL and corresponding date to the lists
                all_urls.append(image_url)
                all_dates.append(date_taken)
                print(f"carousel image added")
                
            # Extract the video URL from the media
            video_versions = media.get('video_versions', [])
            if video_versions:
                video_url = video_versions[0].get('url')
                if video_url:
                    
                    # Add the video URL and corresponding date to the lists
                    all_urls.append(video_url)
                    all_dates.append(date_taken)
                    print(f"carousel video added")

        # Handle cases of unique image, instead of carousel
        image_url = item.get('image_versions2', {}).get('candidates', [{}])[0].get('url')
        if image_url:
            
            # Add the image URL and corresponding date to the lists
            all_urls.append(image_url)
            all_dates.append(date_taken)
            print(f"single image added")

        # Check if 'video_versions' key exists
        video_versions = item.get('video_versions', [])
        if video_versions:
            video_url = video_versions[0].get('url')
            if video_url:
                all_urls.append(video_url)
                all_dates.append(date_taken)
                print(f"video added")
                
# Print or use all collected URLs as needed
print(len(all_urls))
                



0


In [ ]:
# Create a directory to store downloaded files
download_dir = keyword
os.makedirs(download_dir, exist_ok=True)

# Create subfolders for images and videos
image_dir = os.path.join(download_dir, "images")
video_dir = os.path.join(download_dir, "videos")
os.makedirs(image_dir, exist_ok=True)
os.makedirs(video_dir, exist_ok=True)

# Initialize counters for images and videos
image_counter = 1
video_counter = 1

# Iterate through URLs in the all_urls list and download media
for index, url in enumerate(all_urls, 0):
    response = requests.get(url, stream=True)

    # Extract file extension from the URL
    url_path = urlparse(url).path
    file_extension = os.path.splitext(url_path)[1]

    # Determine the file name based on the URL
    if file_extension.lower() in {'.jpg', '.jpeg', '.png', '.gif'}:
        file_name = f"{all_dates[index]}-img-{image_counter}.png"
        destination_folder = image_dir
        image_counter += 1
    elif file_extension.lower() in {'.mp4', '.avi', '.mkv', '.mov'}:
        file_name = f"{all_dates[index]}-vid-{video_counter}.mp4"
        destination_folder = video_dir
        video_counter += 1
    else:
        # Default to the main download directory for other file types
        file_name = f"{all_dates[index]}{file_extension}"
        destination_folder = download_dir

    # Save the file to the appropriate folder
    file_path = os.path.join(destination_folder, file_name)
    
    # Write the content of the response to the file
    with open(file_path, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            if chunk:
                file.write(chunk)

    print(f"Downloaded: {file_path}")

# Print a message indicating the number of downloaded files and the download directory
print(f"Downloaded {len(all_urls)} files to {download_dir}")

Downloaded 0 files to sample-handle
